In [1]:
import pyconll #pip3 install this if you don't have it
import numpy as np
import torchtext.data as tt
import torchtext
import torch 
import torch.nn as nn
import torch.optim as optim
import time

In [25]:
af_vec = torchtext.vocab.FastText(language='af')
nl_vec = torchtext.vocab.FastText(language='nl')

  0%|                                                                                       | 0/871022 [00:00<?, ?it/s]Skipping token b'871022' with 1-dimensional vector [b'300']; likely a header

  2%|█▌                                                                       | 19059/871022 [00:06<04:48, 2957.48it/s]


  5%|███▌                                                                     | 43230/871022 [00:14<04:48, 2873.10it/s]


  7%|█████▍                                                                   | 65060/871022 [00:21<04:13, 3180.12it/s]


 10%|███████▎                                                                 | 86893/871022 [00:28<03:27, 3775.56it/s]


 13%|█████████▋                                                              | 117432/871022 [00:35<03:18, 3805.33it/s]


...

 86%|██████████████████████████████████████████████████████████████▏         | 752467/871022 [03:03<00:29, 3970.04it/s]


 90%|████████████████████████████████████████████████████████████████▌       | 781098/871022 [03:10<00:21, 4228.52it/s]


 93%|███████████████████████████████████████████████████████████████████     | 811258/871022 [03:17<00:13, 4302.11it/s]


 97%|█████████████████████████████████████████████████████████████████████▋  | 842712/871022 [03:24<00:06, 4224.94it/s]


100%|███████████████████████████████████████████████████████████████████████▉| 870975/871022 [03:47<00:00, 4597.59it/s]

In [3]:
embedding_dim = 300

In [26]:
AFRIKAANS_TRAIN = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-train.conllu'
AFRIKAANS_DEV = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-dev.conllu'
AFRIKAANS_TEST = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-test.conllu'

In [27]:
# from https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb
def make_sentences(path):
    data = pyconll.load_from_file(path)
    sentences = []
    tagged_sentences = []
    for each in data:
        tagged_sentence=[]
        sentence = []
        for token in each:
            if token.upos and token.form:
                tagged_sentence.append(token.upos)
                sentence.append(token.form.lower())
        tagged_sentences.append(tagged_sentence)
        sentences.append(sentence)
    return sentences, tagged_sentences

In [28]:
train_afr_raw, tagged_train_afr_raw = make_sentences(AFRIKAANS_TRAIN)
dev_afr_raw, tagged_dev_afr_raw = make_sentences(AFRIKAANS_DEV)
test_afr_raw, tagged_test_afr_raw = make_sentences(AFRIKAANS_TEST)

In [29]:
print("Tagged sentences in train set: ", len(tagged_train_afr_raw))
print("Tagged words in train set:", len([item for sublist in tagged_train_afr_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(tagged_dev_afr_raw))
print("Tagged words in dev set:", len([item for sublist in tagged_dev_afr_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(tagged_test_afr_raw))
print("Tagged words in test set:", len([item for sublist in tagged_test_afr_raw for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(tagged_train_afr_raw)+len(tagged_dev_afr_raw)+len(tagged_dev_afr_raw))

Tagged sentences in train set:  1315
Tagged words in train set: 33894
Tagged sentences in dev set:  194
Tagged words in dev set: 5317
Tagged sentences in test set:  425
Tagged words in test set: 10065
****************************************
Total sentences in dataset: 1703


In [30]:
# from https://github.com/tringm/POSTagger_Pytorch/blob/master/src/util/nlp.py
def build_tag_field(sentences_tokens):
    token_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('tokens', token_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_tokens]
    torch_dataset = tt.Dataset(examples, fields)
    return token_field
    
def build_text_field(sentences_words):
    text_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('text', text_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_words]
    torch_dataset = tt.Dataset(examples, fields)
    return text_field

In [31]:
#fields
train_afr = build_text_field(train_afr_raw)
dev_afr = build_text_field(dev_afr_raw)
test_afr = build_text_field(test_afr_raw)
tagged_train_afr = build_tag_field(tagged_train_afr_raw)
tagged_dev_afr = build_tag_field(tagged_dev_afr_raw)
tagged_test_afr = build_tag_field(tagged_test_afr_raw)

fields_train = (("text", train_afr), ("udtags", tagged_train_afr))
examples_train = [tt.Example.fromlist(item, fields_train) for item in zip(train_afr_raw, tagged_train_afr_raw)]
fields_dev = (("text", dev_afr), ("udtags", tagged_dev_afr))
examples_dev = [tt.Example.fromlist(item, fields_dev) for item in zip(dev_afr_raw, tagged_dev_afr_raw)]
fields_test = (("text", test_afr), ("udtags", tagged_test_afr))
examples_test = [tt.Example.fromlist(item, fields_test) for item in zip(test_afr_raw, tagged_test_afr_raw)]

train_data = tt.Dataset(examples_train, fields_train)
valid_data = tt.Dataset(examples_dev, fields_dev)
test_data = tt.Dataset(examples_test, fields_test)

#build vocabs so that they are shared between splits
train_afr.build_vocab(train_data, valid_data, test_data)
dev_afr.vocab = train_afr.vocab
test_afr.vocab = train_afr.vocab

tagged_train_afr.build_vocab(train_data, valid_data, test_data)
tagged_dev_afr.vocab = tagged_train_afr.vocab
tagged_test_afr.vocab = tagged_train_afr.vocab

In [32]:
matrix_len = len(train_afr.vocab.itos)
weights_matrix = torch.zeros((matrix_len, embedding_dim))
words_found = 0

for i, word in enumerate(train_afr.vocab.itos):
#     print(i, word)
    try: 
        weights_matrix[i] = af_vec[word]
        words_found += 1
    except KeyError:
#         print("test")
        weights_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim, ))

In [33]:
def create_emb_layer(weights_matrix, non_trainable=False):
    input_dim, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(input_dim, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, input_dim, embedding_dim

In [34]:
print(train_afr)

In [35]:
print(train_data)

In [36]:
len(train_data)

1315

In [37]:
# from https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
#model
batch_size=128
device = torch.device('cpu')

#needs to be tuple of dataset objects
train_iterator, valid_iterator, test_iterator = tt.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = batch_size,
    device = device, sort=False)

In [38]:
# try without dropout first
class BiLSTMTagger(nn.Module):
    #https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
    def __init__(self, weights_matrix, hidden_dim, output_dim, n_layers, bidirectional, pad_idx):
        super().__init__()
        self.embedding, input_dim, embedding_dim = create_emb_layer(weights_matrix, True)
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        #fully connected layer
        self.fc = nn.Linear((hidden_dim * 2 if bidirectional else hidden_dim), output_dim)
     
    
    def forward(self, text):
        embedded = self.embedding(text)
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(outputs)
        return predictions

In [39]:
in_dim = len(train_afr.vocab)
emb_dim = 100
hid_dim = 128
out_dim = len(tagged_train_afr.vocab)
n_layers = 2
bidirectional = True
pad_index = train_afr.vocab.stoi[train_afr.pad_token]
tag_pad_idx = tagged_train_afr.vocab.stoi[tagged_train_afr.pad_token]

In [40]:
model = BiLSTMTagger(weights_matrix, hid_dim, out_dim, n_layers, bidirectional, pad_index)
criterion = nn.CrossEntropyLoss(ignore_index = tag_pad_idx)
optimizer = optim.Adam(model.parameters())

In [41]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [42]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        text = batch.text
        tags = batch.udtags
        
        optimizer.zero_grad()       
        predictions = model(text)        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        loss = criterion(predictions, tags) 
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [44]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
N_EPOCHS = 10
torch.device('cpu')

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, tag_pad_idx)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, tag_pad_idx)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 2.685 | Train Acc: 31.73%
	 Val. Loss: 2.340 |  Val. Acc: 27.87%
Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 1.940 | Train Acc: 46.65%
	 Val. Loss: 1.705 |  Val. Acc: 52.75%
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 1.312 | Train Acc: 65.41%
	 Val. Loss: 1.142 |  Val. Acc: 68.92%
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.843 | Train Acc: 76.82%
	 Val. Loss: 0.802 |  Val. Acc: 76.57%
Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 0.577 | Train Acc: 83.47%
	 Val. Loss: 0.622 |  Val. Acc: 81.21%
Epoch: 06 | Epoch Time: 0m 20s
	Train Loss: 0.421 | Train Acc: 88.23%
	 Val. Loss: 0.519 |  Val. Acc: 83.73%
Epoch: 07 | Epoch Time: 0m 20s
	Train Loss: 0.321 | Train Acc: 91.04%
	 Val. Loss: 0.448 |  Val. Acc: 86.37%
Epoch: 08 | Epoch Time: 0m 24s
	Train Loss: 0.251 | Train Acc: 93.16%
	 Val. Loss: 0.394 |  Val. Acc: 87.99%
Epoch: 09 | Epoch Time: 0m 22s
	Train Loss: 0.191 | Train Acc: 95.05%
	 Val. Loss: 0.351 |  Val. Acc: 89.39%
Epoch: 10 | Epoch T

In [46]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, tag_pad_idx)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.309 |  Test Acc: 90.72%
